In [1]:
%matplotlib tk
from Tkinter import TclError
import auto

In [2]:
default_params = {'I': -12, 'gleak': 6.8, 'gKs': 50, 'gKf': 24.1, 'gNa': 100, 'gNaP': 0.8}

# Return a copy of the dict params, coplemented by default values for all parameters that are neither in params nor in exclude.
def add_default_params(params, exclude=None):
    if params is None: 
        params = {}
    else:
        params = params.copy()
        
    if exclude is None: exclude = []    
    for key, val in default_params.items():
        if key not in exclude:
            params.setdefault(key, val)
    return params

add_default_params({'gleak': 10}, exclude=['I'])

{'gKf': 24.1, 'gKs': 50, 'gNa': 100, 'gNaP': 0.8, 'gleak': 10}

In [3]:
# For all these functions, the AUTO constants can be overwritten at any time by using kwargs.

# Integrate the system using the Euler algorithm and return the last point.
def integrate_to_equilibrium(params=None, **kwargs):
    kwargs.setdefault('PAR', add_default_params(params))
    integration = auto.run(e='drosophila', c='integration', **kwargs)
    return integration()[-1]  # last point

# Continue equilibrium forwards and backwards in I from init point.
def continue_equilibrium(init, params=None, **kwargs):
    kwargs.setdefault('PAR', add_default_params(params, exclude=['I']))    
    equilibrium = auto.run(init, c='equilibrium', **kwargs)
    equilibrium += auto.run(init, c='equilibrium', DS='-', **kwargs)
    equilibrium = auto.relabel(auto.merge(equilibrium))  # get single curve with unique labels
    return equilibrium

# Continue limit cycle in I from init point (for example a Hopf bifurcation).
def continue_limit_cycle(init, params=None, **kwargs):
    kwargs.setdefault('PAR', add_default_params(params, exclude=['I']))
    return auto.run(init, c='tube', **kwargs)

# Continue bifurcation point in init for changing continuation_param. 
def continue_in_2_params(init, continuation_param, continuation_param_uzstop=None, params=None, **kwargs):
    kwargs.setdefault('PAR', add_default_params(params, exclude=['I', continuation_param]))
    kwargs.setdefault('ICP', ['I', continuation_param, 'PERIOD'])
    if continuation_param_uzstop is not None:
        kwargs.setdefault('UZSTOP', {'I': [-20, 100], continuation_param: continuation_param_uzstop})
    
    bifurcation_point = auto.run(init, c='2par', **kwargs)
    bifurcation_point += auto.run(init, c='2par', DS='-', **kwargs)
    bifurcation_point = auto.relabel(auto.merge(bifurcation_point))
    return bifurcation_point


# TODO: Make the plot functions simpler.

def plot_bifurcation_diagram(points, saveas=False, **kwargs):
    # AUTO raises a non-critical TclError (bad window path name) on Windows when showing the plot. Prevent its output by wrapping in a try block.
    try:
        auto.plot(points, bifurcation_y=['V', 'MAX V', 'MIN V'], stability=True, use_labels=False, 
                  grid=False, height=350, user_point_symbol='', xlabel='I / pA', ylabel='V / mV', color_list='blue blue green', **kwargs)
    except TclError:
        pass
    
    if saveas:
        p = auto.plot(points, bifurcation_y=['V', 'MAX V', 'MIN V'], stability=True, use_labels=False, 
                      grid=False, height=350, user_point_symbol='', xlabel='I / pA', ylabel='V / mV', color_list='blue blue green', 
                      hide=True, **kwargs)
        p.savefig(saveas, dpi=300)    
        
    
def plot_2_params_diagram(points, second_param, saveas=False, **kwargs):
    # AUTO raises a non-critical TclError (bad window path name) on Windows when showing the plot. Prevent its output by wrapping in a try block.
    try:
        auto.plot(points, bifurcation_x='I', bifurcation_y=second_param, use_labels=False, 
                  grid=False, height=350, xlabel='I / pA', **kwargs)
    except TclError:
        pass
    
    if saveas:
        p = auto.plot(points, bifurcation_x='I', bifurcation_y=second_param, use_labels=False, 
                      grid=False, height=350, xlabel='I / pA', hide=True, **kwargs)
        p.savefig(saveas, dpi=300)
    

Bifurcation diagram for default parameter values

In [4]:
init = integrate_to_equilibrium()

Starting drosophila ...

  BR    PT  TY  LAB       I           MIN V         gleak          gKs           gKf           gNa           gNaP          freq     
   1     1  EP    1  -1.20000E+01  -5.45614E+01   6.80000E+00   5.00000E+01   2.41000E+01   1.00000E+02   8.00000E-01      Infinity
   1     1        2  -1.20000E+01  -5.45614E+01   6.80000E+00   5.00000E+01   2.41000E+01   1.00000E+02   8.00000E-01      Infinity
   1     2        3  -1.20000E+01  -5.45614E+01   6.80000E+00   5.00000E+01   2.41000E+01   1.00000E+02   8.00000E-01      Infinity
   1     3        4  -1.20000E+01  -5.45614E+01   6.80000E+00   5.00000E+01   2.41000E+01   1.00000E+02   8.00000E-01      Infinity
   1     4        5  -1.20000E+01  -5.45614E+01   6.80000E+00   5.00000E+01   2.41000E+01   1.00000E+02   8.00000E-01      Infinity
   1     5  EP    6  -1.20000E+01  -5.45614E+01   6.80000E+00   5.00000E+01   2.41000E+01   1.00000E+02   8.00000E-01      Infinity

 Total Time    0.174E+02
drosophila ... done


In [5]:
equilibrium = continue_equilibrium(init)

Starting drosophila ...

  BR    PT  TY  LAB       I        V                PERIOD         freq         gleak          gKs           gKf           gNa      
   1   651  LP    7  -1.90176E+00  -4.77435E+01   3.69210E+04   2.70848E-05   6.80000E+00   5.00000E+01   2.41000E+01   1.00000E+02
   1  1000        8  -4.94013E+00  -4.17429E+01   3.69210E+04   2.70848E-05   6.80000E+00   5.00000E+01   2.41000E+01   1.00000E+02
   1  1626  LP    9  -1.48988E+01  -3.45427E+01   1.54590E+04   6.46873E-05   6.80000E+00   5.00000E+01   2.41000E+01   1.00000E+02
   1  2000       10  -8.45198E+00  -3.14397E+01   3.77712E+01   2.64752E-02   6.80000E+00   5.00000E+01   2.41000E+01   1.00000E+02
   1  3000       11   1.13559E+01  -2.87468E+01   1.20454E+01   8.30195E-02   6.80000E+00   5.00000E+01   2.41000E+01   1.00000E+02
   1  4000       12   3.12875E+01  -2.71025E+01   9.12536E+00   1.09585E-01   6.80000E+00   5.00000E+01   2.41000E+01   1.00000E+02
   1  5000       13   5.12453E+01  -2.58072E+01   

D:\Python\27_32bit\lib\site-packages\numpy\core\fromnumeric.py:2641: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


drosophila ... done
Starting drosophila ...

  BR    PT  TY  LAB       I        V                PERIOD         freq         gleak          gKs           gKf           gNa      
   1   414  UZ    7  -2.00000E+01  -5.65503E+01   4.60659E+03   2.17080E-04   6.80000E+00   5.00000E+01   2.41000E+01   1.00000E+02

 Total Time    0.328E+00
drosophila ... done
Merge done
Relabeling done


In [6]:
#print equilibrium.summary()
plot_bifurcation_diagram(equilibrium)

Unknown column name: MAX V
Unknown column name: MIN V


In [7]:
tube = continue_limit_cycle(equilibrium('HB1'))

Starting drosophila ...

  BR    PT  TY  LAB       I           MIN V         MAX V        MAX mKs        PERIOD        gleak          gKs           freq     
   9  1000       13   4.08777E+01  -3.23705E+01  -1.77637E+01   3.52584E-01   7.45431E+00   6.80000E+00   5.00000E+01   1.34151E-01
   9  2000       14   2.12608E+01  -3.63629E+01  -1.29058E+01   3.39307E-01   8.67572E+00   6.80000E+00   5.00000E+01   1.15264E-01
   9  3000       15   2.44370E+00  -4.13294E+01  -6.30433E+00   3.25499E-01   1.56343E+01   6.80000E+00   5.00000E+01   6.39619E-02
   9  3591  UZ   16  -2.70771E+00  -4.45008E+01  -6.24261E+00   3.21513E-01   1.00000E+02   6.80000E+00   5.00000E+01   1.00000E-02

 Total Time    0.260E+03
drosophila ... done


In [8]:
plot_bifurcation_diagram(equilibrium + tube, saveas='plots/bifurcation.png')

Created plot


2 parameter continuation for gleak

In [9]:
# Continue saddle-node bifurcation (LP1) for changing gleak (2-parameter continuation)
saddle_node_gleak = continue_in_2_params(equilibrium('LP1'), 'gleak', [0, 20])

Starting drosophila ...

  BR    PT  TY  LAB       I          L2-NORM          V            mKs           mKf           hKf          gleak         PERIOD    
   2   466  BT   13   1.91187E+01   3.91085E+01  -3.90923E+01   2.11143E-01   4.91173E-02   2.71977E-01   8.55953E+00   3.69210E+04
   2   488  CP   14   1.98699E+01   3.83728E+01  -3.83564E+01   2.17366E-01   5.40669E-02   2.48380E-01   8.60603E+00   3.69210E+04
   2  1000       15  -5.20240E+00   3.50656E+01  -3.50477E+01   2.46959E-01   8.26534E-02   1.59933E-01   7.27974E+00   3.69210E+04
   2  1297  UZ   16  -2.00000E+01   3.43283E+01  -3.43099E+01   2.53914E-01   9.06811E-02   1.44095E-01   6.55207E+00   3.69210E+04

 Total Time    0.261E+01
drosophila ... done
Starting drosophila ...

  BR    PT  TY  LAB       I          L2-NORM          V            mKs           mKf           hKf          gleak         PERIOD    
   2  1000       13  -5.12107E-02   8.49357E+01  -8.49181E+01   2.60919E-02   9.45218E-05   9.98712E-01   1.51

In [10]:
# Continue Hopf bifurcation (HB1) for changing gleak (2-parameter continuation)
hopf_gleak = continue_in_2_params(equilibrium('HB1'), 'gleak', UZSTOP={'I': [-20, 170], 'gleak': [0, 20]})

Starting drosophila ...

  BR    PT  TY  LAB       I          L2-NORM          V            mKs           mKf           hKf          gleak         PERIOD    
   9  1000       13   1.09456E+02   2.72228E+01  -2.71959E+01   3.27275E-01   2.09688E-01   4.89226E-02   9.65797E+00   8.47888E+00
   9  2000       14   1.40000E+02   3.25089E+01  -3.24891E+01   2.71625E-01   1.13559E-01   1.10548E-01   1.35651E+01   1.30071E+01
   9  3000       15   9.01186E+01   3.55486E+01  -3.55310E+01   2.42472E-01   7.77502E-02   1.71055E-01   1.21708E+01   1.92242E+01
   9  4000       16   4.02337E+01   3.79568E+01  -3.79403E+01   2.20942E-01   5.70700E-02   2.35660E-01   9.79956E+00   3.67259E+01
   9  4434  BT   17   1.91187E+01   3.91085E+01  -3.90923E+01   2.11143E-01   4.91173E-02   2.71977E-01   8.55953E+00   2.59459E+03

 Total Time    0.889E+01
drosophila ... done
Starting drosophila ...

  BR    PT  TY  LAB       I          L2-NORM          V            mKs           mKf           hKf          gle

In [11]:
plot_2_params_diagram(saddle_node_gleak + hopf_gleak, 'gleak', ylabel='gleak / nS', miny=0, saveas='plots/2par_gleak.png')

Created plot


Bifurcation diagram for high gleak

In [12]:
# As can be seen in the plot above, the system is governed by Hopf bifurcations for gleak >~ 8.6
# Make a bifurcation diagram for a high gleak value to see these bifurcations
new_gleak = 8.84
params = {'gleak': new_gleak}
init_gleak = integrate_to_equilibrium(params=params)

Starting drosophila ...

  BR    PT  TY  LAB       I           MIN V         gleak          gKs           gKf           gNa           gNaP          freq     
   1     1  EP    1  -1.20000E+01  -5.45614E+01   8.84000E+00   5.00000E+01   2.41000E+01   1.00000E+02   8.00000E-01      Infinity
   1     1        2  -1.20000E+01  -5.47226E+01   8.84000E+00   5.00000E+01   2.41000E+01   1.00000E+02   8.00000E-01      Infinity
   1     2        3  -1.20000E+01  -5.47226E+01   8.84000E+00   5.00000E+01   2.41000E+01   1.00000E+02   8.00000E-01      Infinity
   1     3        4  -1.20000E+01  -5.47226E+01   8.84000E+00   5.00000E+01   2.41000E+01   1.00000E+02   8.00000E-01      Infinity
   1     4        5  -1.20000E+01  -5.47226E+01   8.84000E+00   5.00000E+01   2.41000E+01   1.00000E+02   8.00000E-01      Infinity
   1     5  EP    6  -1.20000E+01  -5.47226E+01   8.84000E+00   5.00000E+01   2.41000E+01   1.00000E+02   8.00000E-01      Infinity

 Total Time    0.166E+02
drosophila ... done


In [13]:
equilibrium_gleak = continue_equilibrium(init_gleak, params=params, UZSTOP={'I': [-20, 170]})

Starting drosophila ...

  BR    PT  TY  LAB       I        V                PERIOD         freq         gleak          gKs           gKf           gNa      
   1  1000        7   7.40261E+00  -5.00543E+01   4.62893E+02   2.16033E-03   8.84000E+00   5.00000E+01   2.41000E+01   1.00000E+02
   1  2000        8   2.35478E+01  -3.92051E+01   7.60091E+01   1.31563E-02   8.84000E+00   5.00000E+01   2.41000E+01   1.00000E+02
   1  2020  HB    9   2.36504E+01  -3.88291E+01   7.97427E+01   1.25403E-02   8.84000E+00   5.00000E+01   2.41000E+01   1.00000E+02
   1  3000       10   4.03951E+01  -3.13236E+01   1.62848E+01   6.14068E-02   8.84000E+00   5.00000E+01   2.41000E+01   1.00000E+02
   1  4000       11   6.02724E+01  -2.91377E+01   1.09964E+01   9.09392E-02   8.84000E+00   5.00000E+01   2.41000E+01   1.00000E+02
   1  5000       12   8.02143E+01  -2.76198E+01   9.03512E+00   1.10679E-01   8.84000E+00   5.00000E+01   2.41000E+01   1.00000E+02
   1  5796  HB   13   9.60913E+01  -2.66255E+01   

In [14]:
tube_gleak = continue_limit_cycle(equilibrium_gleak('HB1'), params=params, 
                             UZSTOP={'I': [-20, 170], 'PERIOD': [0, 100]}, STOP=['LP2'], ILP=1)  # ILP=1 to detect folds

Starting drosophila ...

  BR    PT  TY  LAB       I           MIN V         MAX V        MAX mKs        PERIOD        gleak          gKs           freq     
   4   266  LP   14   2.35240E+01  -4.31876E+01  -1.90110E+01   3.24458E-01   9.06429E+01   8.84000E+00   5.00000E+01   1.10323E-02
   4  1000       15   3.41983E+01  -4.05086E+01  -1.13383E+01   3.16432E-01   1.49891E+01   8.84000E+00   5.00000E+01   6.67152E-02
   4  2000       16   5.36619E+01  -3.69716E+01  -1.46409E+01   3.30015E-01   9.71710E+00   8.84000E+00   5.00000E+01   1.02911E-01
   4  3000       17   7.33957E+01  -3.37839E+01  -1.83142E+01   3.41949E-01   8.33724E+00   8.84000E+00   5.00000E+01   1.19944E-01
   4  4000       18   9.30377E+01  -2.93669E+01  -2.36143E+01   3.42290E-01   8.10364E+00   8.84000E+00   5.00000E+01   1.23401E-01
   4  5000  EP   19   8.09800E+01  -3.24495E+01  -1.98911E+01   3.44710E-01   8.14551E+00   8.84000E+00   5.00000E+01   1.22767E-01

 Total Time    0.363E+03
drosophila ... done


In [15]:
plot_bifurcation_diagram(equilibrium_gleak + tube_gleak, saveas='plots/bifurcation_gleak={0}.png'.format(new_gleak))

Created plot


2 parameter continuation for gKs

In [16]:
saddle_node_gKs = continue_in_2_params(equilibrium('LP1'), 'gKs', [0, 300])

Starting drosophila ...

  BR    PT  TY  LAB       I          L2-NORM          V            mKs           mKf           hKf           gKs          PERIOD    
   2  1000       13  -1.06131E+00   4.67175E+01  -4.66956E+01   1.54474E-01   1.78275E-02   5.70185E-01   9.98611E+01   3.69210E+04
   2  2000       14   1.91601E-01   4.36073E+01  -4.35904E+01   1.75959E-01   2.70696E-02   4.41536E-01   1.49739E+02   3.69210E+04
   2  2541  BT   15   1.36602E+00   4.07594E+01  -4.07436E+01   1.97660E-01   3.95318E-02   3.29731E-01   1.76543E+02   3.69210E+04
   2  2548  CP   16   1.37925E+00   4.05250E+01  -4.05091E+01   1.99534E-01   4.07746E-02   3.21153E-01   1.76761E+02   3.69210E+04
   2  3000       17  -4.78563E-01   3.82280E+01  -3.82116E+01   2.18606E-01   5.50946E-02   2.43902E-01   1.54489E+02   3.69210E+04
   2  4000       18  -6.22347E+00   3.62728E+01  -3.62556E+01   2.35850E-01   7.08976E-02   1.88865E-01   1.04862E+02   3.69210E+04
   2  5000       19  -1.39123E+01   3.47283E+01  -

In [17]:
hopf_gKs = continue_in_2_params(equilibrium('HB1'), 'gKs', UZSTOP={'I': [-20, 300], 'gKs': [0, 1300]})

Starting drosophila ...

  BR    PT  TY  LAB       I          L2-NORM          V            mKs           mKf           hKf           gKs          PERIOD    
   9  1000       13   8.45967E+01   2.57904E+01  -2.57606E+01   3.43339E-01   2.44274E-01   3.89196E-02   9.32063E+01   7.04228E+00
   9  2000       14   1.06712E+02   2.62364E+01  -2.62075E+01   3.38296E-01   2.33106E-01   4.18033E-02   1.38038E+02   6.79209E+00
   9  3000       15   1.26175E+02   2.66775E+01  -2.66495E+01   3.33345E-01   2.22415E-01   4.48555E-02   1.84086E+02   6.61264E+00
   9  4000       16   1.43215E+02   2.71091E+01  -2.70819E+01   3.28536E-01   2.12297E-01   4.80467E-02   2.31085E+02   6.48200E+00
   9  5000       17   1.58058E+02   2.75296E+01  -2.75032E+01   3.23885E-01   2.02768E-01   5.13621E-02   2.78825E+02   6.38684E+00
   9  6000       18   1.70910E+02   2.79391E+01  -2.79134E+01   3.19391E-01   1.93801E-01   5.47968E-02   3.27140E+02   6.31866E+00
   9  7000       19   1.81953E+02   2.83383E+01  -

In [18]:
plot_2_params_diagram(saddle_node_gKs + hopf_gKs, 'gKs', ylabel='gKs / nS', miny=0, saveas='plots/2par_gKs.png')  #, minx=-20, maxx=120, miny=0, maxy=500)

Created plot


2 parameter continuation for gKf

In [19]:
saddle_node_gKf = continue_in_2_params(equilibrium('LP1'), 'gKf', [0, 500])
saddle_node_gKf += continue_in_2_params(equilibrium('LP2'), 'gKf', [0, 500])

Starting drosophila ...

  BR    PT  TY  LAB       I          L2-NORM          V            mKs           mKf           hKf           gKf          PERIOD    
   2  1000       13  -1.90170E+00   4.77662E+01  -4.77434E+01   1.47725E-01   1.54718E-02   6.12356E-01   7.39800E+01   3.69210E+04
   2  2000       14  -1.90165E+00   4.77661E+01  -4.77432E+01   1.47726E-01   1.54721E-02   6.12351E-01   1.23980E+02   3.69210E+04
   2  3000       15  -1.90159E+00   4.77659E+01  -4.77431E+01   1.47727E-01   1.54723E-02   6.12345E-01   1.73980E+02   3.69210E+04
   2  4000       16  -1.90153E+00   4.77658E+01  -4.77430E+01   1.47728E-01   1.54726E-02   6.12340E-01   2.23980E+02   3.69210E+04
   2  5000       17  -1.90147E+00   4.77657E+01  -4.77428E+01   1.47729E-01   1.54729E-02   6.12335E-01   2.73980E+02   3.69210E+04
   2  6000       18  -1.90141E+00   4.77655E+01  -4.77427E+01   1.47730E-01   1.54732E-02   6.12329E-01   3.23980E+02   3.69210E+04
   2  7000       19  -1.90136E+00   4.77654E+01  -

In [20]:
hopf_gKf = continue_in_2_params(equilibrium('HB1'), 'gKf', UZSTOP={'I': [-20, 170], 'gKf': [0, 500]})

Starting drosophila ...

  BR    PT  TY  LAB       I          L2-NORM          V            mKs           mKf           hKf           gKf          PERIOD    
   9  1000       13   5.99932E+01   2.53564E+01  -2.53257E+01   3.48281E-01   2.55488E-01   3.62967E-02   7.39791E+01   7.38686E+00
   9  2000       14   6.02905E+01   2.53622E+01  -2.53314E+01   3.48215E-01   2.55337E-01   3.63304E-02   1.23978E+02   7.37278E+00
   9  3000       15   6.05865E+01   2.53679E+01  -2.53372E+01   3.48149E-01   2.55185E-01   3.63642E-02   1.73977E+02   7.35886E+00
   9  4000       16   6.08811E+01   2.53737E+01  -2.53430E+01   3.48083E-01   2.55034E-01   3.63981E-02   2.23977E+02   7.34508E+00
   9  5000       17   6.11744E+01   2.53795E+01  -2.53488E+01   3.48017E-01   2.54882E-01   3.64321E-02   2.73976E+02   7.33144E+00
   9  6000       18   6.14663E+01   2.53853E+01  -2.53547E+01   3.47951E-01   2.54730E-01   3.64661E-02   3.23975E+02   7.31794E+00
   9  7000       19   6.17570E+01   2.53911E+01  -

In [21]:
plot_2_params_diagram(saddle_node_gKf + hopf_gKf, 'gKf', ylabel='gKf / nS', color_list='black black red', saveas='plots/2par_gKf.png')

Created plot


2 parameter continuation for gNa

In [22]:
saddle_node_gNa = continue_in_2_params(equilibrium('LP1'), 'gNa', [0, 1000])

Starting drosophila ...

  BR    PT  TY  LAB       I          L2-NORM          V            mKs           mKf           hKf           gNa          PERIOD    
   2   458  CP   13   2.66731E+00   3.89877E+01  -3.89714E+01   2.12156E-01   4.98996E-02   2.68007E-01   8.02285E+01   3.69210E+04
   2   460  BT   14   2.66628E+00   3.89351E+01  -3.89188E+01   2.12598E-01   5.02440E-02   2.66290E-01   8.02303E+01   3.69210E+04
   2  1000       15  -1.46904E+01   3.45787E+01  -3.45605E+01   2.51537E-01   8.78787E-02   1.49323E-01   9.97932E+01   3.69210E+04
   2  1149  UZ   16  -2.00000E+01   3.41719E+01  -3.41534E+01   2.55406E-01   9.24714E-02   1.40908E-01   1.04974E+02   3.69210E+04

 Total Time    0.245E+01
drosophila ... done
Starting drosophila ...

  BR    PT  TY  LAB       I          L2-NORM          V            mKs           mKf           hKf           gNa          PERIOD    
   2  1000       13  -5.33816E+00   4.99841E+01  -4.99606E+01   1.34247E-01   1.14513E-02   6.95671E-01   1.49

In [23]:
hopf_gNa = continue_in_2_params(equilibrium('HB1'), 'gNa', [0, 1000])

Starting drosophila ...

  BR    PT  TY  LAB       I          L2-NORM          V            mKs           mKf           hKf           gNa          PERIOD    
   9  1000       13   4.66292E+01   3.11026E+01  -3.10813E+01   2.85838E-01   1.34557E-01   8.94975E-02   5.74608E+01   1.23553E+01
   9  1168  GH   14   3.85218E+01   3.25768E+01  -3.25570E+01   2.70950E-01   1.12621E-01   1.11667E-01   5.62634E+01   1.42989E+01
   9  2000       15   4.77340E+00   3.85150E+01  -3.84986E+01   2.16153E-01   5.30751E-02   2.52831E-01   7.69274E+01   6.65111E+01
   9  2074  GH   16   2.81364E+00   3.89054E+01  -3.88891E+01   2.12848E-01   5.04393E-02   2.65323E-01   7.99854E+01   2.52999E+02
   9  2090  BT   17   2.66628E+00   3.89351E+01  -3.89188E+01   2.12598E-01   5.02440E-02   2.66290E-01   8.02303E+01   7.39987E+03

 Total Time    0.461E+01
drosophila ... done
Starting drosophila ...

  BR    PT  TY  LAB       I          L2-NORM          V            mKs           mKf           hKf           gN

In [24]:
plot_2_params_diagram(saddle_node_gNa + hopf_gNa, 'gNa', ylabel='gNa / nS', saveas='plots/2par_gNa.png')

Created plot


2 parameter continuation for gNaP

In [25]:
saddle_node_gNaP = continue_in_2_params(equilibrium('LP1'), 'gNaP', UZSTOP={'I': [-50, 100], 'gNaP': [-10, 10]})
saddle_node_gNaP += continue_in_2_params(equilibrium('LP2'), 'gNaP', UZSTOP={'I': [-50, 100], 'gNaP': [-10, 10]}) 

Starting drosophila ...

  BR    PT  TY  LAB       I          L2-NORM          V            mKs           mKf           hKf           gNaP         PERIOD    
   2  1000       13   4.71435E+01   4.16302E+01  -4.16144E+01   1.90815E-01   3.52264E-02   3.62561E-01   1.10684E-01   3.69210E+04
   2  2000       14   9.71261E+01   4.05146E+01  -4.04987E+01   1.99618E-01   4.08309E-02   3.20774E-01  -5.40989E-01   3.69210E+04
   2  2058  UZ   15   1.00000E+02   4.04703E+01  -4.04544E+01   1.99974E-01   4.10701E-02   3.19168E-01  -5.78142E-01   3.69210E+04

 Total Time    0.358E+01
drosophila ... done
Starting drosophila ...

  BR    PT  TY  LAB       I          L2-NORM          V            mKs           mKf           hKf           gNaP         PERIOD    
   2   993  UZ   13  -5.00000E+01   5.82786E+01  -5.82551E+01   9.27493E-02   3.68771E-03   9.01071E-01   3.76605E+00   3.69210E+04

 Total Time    0.205E+01
drosophila ... done
Merge done
Relabeling done
Starting drosophila ...

  BR    PT  

In [26]:
hopf_gNaP = continue_in_2_params(equilibrium('HB1'), 'gNaP', UZSTOP={'I': [-50, 100], 'gNaP': [-10, 10]})

Starting drosophila ...

  BR    PT  TY  LAB       I          L2-NORM          V            mKs           mKf           hKf           gNaP         PERIOD    
   9   809  UZ   13   1.00000E+02   2.50527E+01  -2.50213E+01   3.51759E-01   2.63533E-01   3.45633E-02   3.01072E-01   7.23482E+00

 Total Time    0.139E+01
drosophila ... done
Starting drosophila ...

  BR    PT  TY  LAB       I          L2-NORM          V            mKs           mKf           hKf           gNaP         PERIOD    
   9  1000       13   9.82053E+00   2.57229E+01  -2.56930E+01   3.44105E-01   2.45996E-01   3.85000E-02   1.41453E+00   7.61300E+00
   9  2000       14  -4.01742E+01   2.61013E+01  -2.60722E+01   3.39820E-01   2.36451E-01   4.09089E-02   2.02722E+00   7.83385E+00
   9  2197  UZ   15  -5.00000E+01   2.61764E+01  -2.61474E+01   3.38972E-01   2.34587E-01   4.14039E-02   2.14724E+00   7.87841E+00

 Total Time    0.369E+01
drosophila ... done
Merge done
Relabeling done


In [27]:
plot_2_params_diagram(saddle_node_gNaP + hopf_gNaP, 'gNaP', ylabel='gNaP / nS', color_list='black black red', saveas='plots/2par_gNaP.png')

Created plot
